In [1]:
import pandas as pd
import os
import re 

"tot_masks_trained": Total number of masks used in training

"num_masks": number of the "top-k" masks used when evaluating

"dropout_probs1": Dropout probability on layer 1

"dropout_probs2": Dropout probability on layer 2

"num_groups": Number of partitioned groups for masks to train off of

"increment_amt": amount to increment each mask when using MCMC

In [2]:
column_names = ["tot_masks_trained", "num_masks",
    "test_accuracy",
    "EPOCHS",
    "BATCH_SIZE",
    "LR",
    "dropout_probs1",
    "dropout_probs2",
    "num_groups",
    "increment_amt"]
df = pd.DataFrame(columns=column_names)

In [3]:
for folder in os.listdir("experiments"):
    path = os.path.join("experiments", folder)
    # adding this because I forgot to add number of masks trained on in the csv
    match = re.findall(r'\d+', folder.split("_")[0])[-1] 
    if "results.csv" not in os.listdir(path=path):
        continue
    new_df = pd.read_csv(os.path.join(path, "results.csv"))
    new_df["tot_masks_trained"] = match
    df = pd.concat([df, new_df])


/var/folders/5q/7b4nn19d1rj3vsg11yxvc8lw0000gn/T/ipykernel_3141/2272338392.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_df])


High dropout probabilities do have the ability of yielding stronger performance. It would seem; however, there is a caveat that there must be fewer masks (but not necessarily only one) that we train on in general. 

I would hypotheisize that too many independent masks trying to train on their own end up adding too much noise to weights and don't allow them to learn meaningful representations.

For this particular architecture (hidden layer size (256, 128)) it seems that 3 masks yields results that are somewhat close to a standard deep-net which uses all neurons (i.e dropout probabiltieies of [0.0, 0.0])

In [8]:
df[(df['test_accuracy'] > 0.89) & (df['dropout_probs1'] >= 0.7) & \
   (df['dropout_probs2'] >= 0.4)].sort_values(by="test_accuracy").tail(20)

,tot_masks_trained,num_masks,test_accuracy,EPOCHS,BATCH_SIZE,LR,dropout_probs1,dropout_probs2,num_groups,increment_amt
3,3,3,0.8963,12,32,0.001,0.7,0.7,1,10
3,3,3,0.8963,12,32,0.001,0.7,0.7,1,50
3,3,3,0.8963,12,32,0.001,0.7,0.7,1,1
2,3,2,0.9017,12,32,0.001,0.7,0.4,1,1
2,3,2,0.9017,12,32,0.001,0.7,0.4,1,50
2,3,2,0.9017,12,32,0.001,0.7,0.4,1,10
3,3,3,0.9085,12,32,0.001,0.7,0.4,1,1
3,3,3,0.9085,12,32,0.001,0.7,0.4,1,50
3,3,3,0.9085,12,32,0.001,0.7,0.4,1,10


In [9]:
df[(df['test_accuracy'] > 0.89)].sort_values(by="test_accuracy").tail(20)

,tot_masks_trained,num_masks,test_accuracy,EPOCHS,BATCH_SIZE,LR,dropout_probs1,dropout_probs2,num_groups,increment_amt
2,3,2,0.9057,12,32,0.001,0.2,0.4,1,1
3,3,3,0.9065,12,32,0.001,0.4,0.2,1,50
3,3,3,0.9065,12,32,0.001,0.4,0.2,1,10
3,3,3,0.9065,12,32,0.001,0.4,0.2,1,1
2,3,2,0.9080,12,32,0.001,0.4,0.4,1,1
2,3,2,0.9080,12,32,0.001,0.4,0.4,1,50
2,3,2,0.9080,12,32,0.001,0.4,0.4,1,10
2,3,2,0.9084,12,32,0.001,0.4,0.2,1,1
2,3,2,0.9084,12,32,0.001,0.4,0.2,1,50
2,3,2,0.9084,12,32,0.001,0.4,0.2,1,10
